### **Download Sentiment Dataset**

In [ ]:
! pip install --quiet datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is in

In [ ]:
from datasets import load_dataset

sentiment_dataset = load_dataset("rasyosef/amharic-sentiment")
sentiment_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


Generating train split:   0%|          | 0/2223 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/279 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 2223
    })
    dev: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 279
    })
    test: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 279
    })
})

In [ ]:
sentiment_dataset["train"].features

{'clean_tweet': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [ ]:
label_names = sentiment_dataset["train"].features["label"].names
label_names

['negative', 'positive']

### **Tokenize Tweets**

In [ ]:
from transformers import AutoTokenizer

model_id = "rasyosef/bert-medium-amharic"
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(tokenizer.tokenize("ከሀገራቸው ከኢትዮጵያ ከወጡ ግማሽ ምዕተ [MASK] ተቆጥሯል።"))

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/274k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/725k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

['ከሀ', '##ገራቸው', 'ከኢትዮጵያ', 'ከወጡ', 'ግማሽ', 'ምዕተ', '[MASK]', 'ተቆጥ', '##ሯል', '።']


In [ ]:
def tokenize_dataset(samples):
  tokenized_samples = tokenizer(samples["clean_tweet"], truncation=True, max_length=512)
  return tokenized_samples

tokenize_dataset({"clean_tweet":["ከሀገራቸው ከኢትዮጵያ ከወጡ ግማሽ"]})

{'input_ids': [[2, 12051, 11275, 10894, 27845, 13971, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
preprocessed_datasets = sentiment_dataset.map(
    tokenize_dataset,
    batched=True,
    remove_columns=["clean_tweet"]
  )
preprocessed_datasets

Map:   0%|          | 0/2223 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2223
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 279
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 279
    })
})

### **Load Amharic BERT model**

In [ ]:
id2label={i: label for i, label in enumerate(label_names)}
label2id={label: i for i, label in enumerate(label_names)}

print(id2label, label2id)

{0: 'negative', 1: 'positive'} {'negative': 0, 'positive': 1}


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
    # device_map="cuda"
)

config.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/162M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rasyosef/bert-medium-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### **Finetuning**

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [ ]:
from transformers import TrainingArguments

batch_size = 32
epochs = 8

training_args = TrainingArguments(
    output_dir=model_id+"-finetuned",
    learning_rate=2e-5,
    #lr_scheduler_type="linear",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    seed=42,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric1 = evaluate.load("accuracy")
  metric2 = evaluate.load("precision")
  metric3 = evaluate.load("recall")
  metric4 = evaluate.load("f1")

  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
  precision = metric2.compute(predictions=predictions, references=labels, average='macro')["precision"]
  recall = metric3.compute(predictions=predictions, references=labels, average='macro')["recall"]
  f1 = metric4.compute(predictions=predictions, references=labels, average='macro')["f1"]

  return {
      "accuracy": accuracy,
      "precision": precision,
      "recall": recall,
      "f1": f1
  }

compute_metrics(([[1,0], [0,1]], [0,1]))

{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=preprocessed_datasets["train"],
    eval_dataset=preprocessed_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6914638876914978,
 'eval_accuracy': 0.5232974910394266,
 'eval_precision': 0.49756493506493504,
 'eval_recall': 0.4986046511627907,
 'eval_f1': 0.44975310289603626,
 'eval_runtime': 4.3375,
 'eval_samples_per_second': 64.323,
 'eval_steps_per_second': 2.075}

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.608800,0.467424,0.799283,0.798306,0.797597,0.797910
2,0.438300,0.416620,0.831541,0.835885,0.826512,0.828689
3,0.375700,0.422759,0.827957,0.831575,0.823178,0.825206
4,0.323000,0.428638,0.838710,0.840094,0.835349,0.836823
5,0.287400,0.487095,0.817204,0.833268,0.808295,0.811139
6,0.270300,0.459537,0.831541,0.833667,0.827597,0.829296
7,0.234800,0.474566,0.831541,0.834700,0.827054,0.829002
8,0.230600,0.473802,0.835125,0.836045,0.832016,0.833325


TrainOutput(global_step=560, training_loss=0.34611972059522356, metrics={'train_runtime': 106.2487, 'train_samples_per_second': 167.381, 'train_steps_per_second': 5.271, 'total_flos': 174889918755732.0, 'train_loss': 0.34611972059522356, 'epoch': 8.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.4286377727985382,
 'eval_accuracy': 0.8387096774193549,
 'eval_precision': 0.8400943396226415,
 'eval_recall': 0.8353488372093023,
 'eval_f1': 0.836823020236285,
 'eval_runtime': 2.1099,
 'eval_samples_per_second': 132.231,
 'eval_steps_per_second': 4.266,
 'epoch': 8.0}

### **Evaluate Model**

In [ ]:
import torch

num_samples = len(preprocessed_datasets["test"])
inputs = data_collator([preprocessed_datasets["test"][i] for i in range(num_samples)])
# print(inputs)

#output = model(**inputs)
output = model(**{k: v.to("cuda") for k, v in inputs.items()})
y_pred = np.argmax(output.logits.tolist(), axis=-1)
y_test = np.array(preprocessed_datasets["test"]["label"])

print(y_pred)
print(y_test)

[1 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0 1 1 1 1 0 1 1 0 0 1
 0 1 1 0 0 0 1 1 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0 0
 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 0 1
 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 0 1 1 1
 0 1 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0
 1 0 1 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1
 0 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0
 1 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0]
[1 1 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1
 0 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0
 0 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 0 1 1 1 1 0 1
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 1 1 0
 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 1 0
 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 1 0

In [ ]:
from sklearn import metrics

metrics.confusion_matrix(y_test, y_pred)

array([[132,  18],
       [ 30,  99]])

In [ ]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.88      0.85       150
           1       0.85      0.77      0.80       129

    accuracy                           0.83       279
   macro avg       0.83      0.82      0.83       279
weighted avg       0.83      0.83      0.83       279



### **Test in Pipeline**

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
pipe("ቆንጆ ፊልም ነው")

[{'label': 'positive', 'score': 0.973303496837616}]

In [ ]:
pipe("አሪፍ ልብስ")

[{'label': 'positive', 'score': 0.9282798767089844}]

In [ ]:
pipe("ጅላንፎ")

[{'label': 'negative', 'score': 0.854489266872406}]

In [ ]:
pipe("እዚህ ሰፈር በማታ መምጣት እፈራለሁ")

[{'label': 'negative', 'score': 0.5514835715293884}]

In [ ]:
pipe("ዩክረን እና ሩስያ ከባድ ውግያ ላይ ናቸው")

[{'label': 'negative', 'score': 0.7378876805305481}]

**Mis-classified**

In [ ]:
misclassified = []
for i in range(len(y_pred)):
  if y_pred[i] != y_test[i]:
    misclassified.append(i)

print(misclassified)

[2, 12, 13, 24, 27, 29, 33, 49, 53, 54, 59, 65, 72, 81, 89, 99, 100, 101, 102, 103, 106, 107, 115, 120, 125, 144, 147, 154, 162, 179, 186, 201, 203, 205, 207, 210, 211, 212, 220, 233, 239, 240, 243, 249, 261, 263, 269, 270]


In [ ]:
for idx in misclassified:
  input_ids = preprocessed_datasets["test"][idx]["input_ids"]
  label = preprocessed_datasets["test"][idx]["label"]
  print(label, tokenizer.decode(input_ids))

0 [CLS] : ለማ መገርሳ ሆይ! የቀድሞ ወዳጅህ ናሁሰናይ ከመቐለ ይህን ባለ 6 ነጥብ መልእክት ልኮልሃል ፦ 1. ጠባቂዎችህን ሙሉ በሙሉ ከአብይ ንኪኪ እና ትእዛዝ አፅዳ ፡ ፡ የጥበቃ ጥንካሬው ሶስት እጥፍ ጨምር ፡ … [SEP]
1 [CLS] እንዲህ አይነት ርዕስ ጽፎ ለበዐል ይጠብቁን ድንግርግር ይላል ግን አሻም በበዓልም አልቀልድም ካለ ምን ይደረግ? 😃 ስሜነህ አያሌውና ሱራፌል ሲጠይቁ እንኳን ብንሰማ ይሻለናል በውነት ። [SEP]
0 [CLS] ከእንደ አንተ አይነቱ ደደብ መሀይም ጭራቅ የምንላቀቅበት ዘመን ይሁን!!! ሁሌም አብየየየየ [SEP]
1 [CLS] ዛሬ እንደ እኔ ምህረት የበዛለት ማን ነው??? [SEP]
0 [CLS] " ህዉሃት ዲያብሎስ ነዉ ። ከዲያብሎስ ጋር የሚተባበርም ዲያብሎስ ብቻ ነዉ ። " [SEP]
0 [CLS] ያሰብከው ሁሉ ይሳካ! በእውነትና እውቀትና በተመሰረትርው ራዕይ ፓርቲ ኢትዮጵያና ህዝቦችዋ ታፍረውና ተከውረው ይኖራሉ ። [SEP]
0 [CLS] አጋንንቶች ነፍሴን ይመገባሉ ። መሪዬን አገለግላለሁ ፡ ፡ እኔ ነፍሳትን አመጣለሁ [SEP]
0 [CLS] በመካከላችን ማንም የለም ፣ በገዛ እጄ ከገደለው ፍጥረት ሁሉ ጋር እኖራለሁ ምንም እንኳን ከእንግዲህ ከእኛ ጋር ባይሆንም [SEP]
0 [CLS] ታሪካችን ፡ እንደገና ፡ ተበዉዞ ፡ መስራት ፡ አለበት ፡ እስካሁን ፡ ያለው ፡ የኣንድ ፡ ወገን ፡ ታሪክ ፡ ነው ። [SEP]
0 [CLS] : በተስፋ የምትጠብቅ እናት በሠላም የላከችው ልጇ ሬሳው በሳጥን ተጭኖ ሲመጣላት የሚሰማትን የሀዘን ስሜት ከባድ ነው ስለዚህ እማራለሁ ብለሁ ብለው ሄደው እሬሳቸው ለቤተ … [SEP]
0 [CLS] ከሀገሬው ይልቅ ለቻይናዉያን ምቹ የሆነች ሀገር [UNK]... [SEP]
1 [CLS] ኦሮምያ የደጋገዎች አገር ናት እን